In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict 
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState

import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

llm = init_chat_model("openai:gpt-4o-mini", openai_api_key=api_key)

In [ ]:
class State(MessagesState):
    custom_stuff: str

graph_builder = StateGraph(State)

In [ ]:
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    return {
        "messages": [response]
    }

In [ ]:
graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)


graph = graph_builder.compile()

graph.invoke(
    {
        "messages": [
            {"role": "user", "content": "Hello, how are you?"}
        ]
    }
)

{'messaages': [HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}, id='ecfecaa5-216d-44df-9d37-127f0476f5d7'),
  AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 13, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-COlJjGQpbwKeD56qYFiGxhWARLHkM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--fd5a7132-b6ec-42ce-8988-a42f013255dd-0', usage_metadata={'input_tokens': 13, 'output_tokens': 30, 'total_tokens': 43, 'input_token_details': {'audio': 0,